# Evaluation  for MovingMNIST

In [1]:
%matplotlib inline

import os
import sys
sys.path.append(os.path.expanduser("~/libs"))

import cv2
import numpy as np
import tensorflow as tf
import tensortools as tt

from model.frame_prediction import LSTMConv2DPredictionModel

In [ ]:
INPUT_SEQ_LENGTH = 10
OUTPUT_SEQ_LENGTH = 10

In [ ]:
# validation while training
OUT_DIR_NAME = "out-eval"
NUM_SAMPLES = 4
GIF_FPS = 5

#### Directory Paths:

In [2]:
ROOT_DIR = "/work/sauterme/"
DATA_DIR = ROOT_DIR + "data"

In [3]:
TRAIN_DIR = "..."

assert os.path.exists(TRAIN_DIR)

AssertionError: 

### Data

In [ ]:
dataset_valid = tt.datasets.moving_mnist.MovingMNISTValidDataset(DATA_DIR,
                                                                 input_shape=[INPUT_SEQ_LENGTH, 64, 64, 1],
                                                                 target_shape=[OUTPUT_SEQ_LENGTH, 64, 64, 1],
                                                                 as_binary=True)
dataset_test = tt.datasets.moving_mnist.MovingMNISTTestDataset(DATA_DIR,
                                                               input_shape=[INPUT_SEQ_LENGTH, 64, 64, 1],
                                                               target_shape=[OUTPUT_SEQ_LENGTH, 64, 64, 1],
                                                               as_binary=True)

### Runtime

In [4]:
GPU_ID = 0

In [5]:
runtime = tt.core.DefaultRuntime(train_dir=TRAIN_DIR, gpu_devices=[GPU_ID])

In [ ]:
runtime.register_datasets(None, dataset_valid, dataset_valid)

Consider to restore the **EMA variables** as well when building the model. These might generate worse results in models using batch-normalization, since the shaddow variables might get restored properly...

In [ ]:
runtime.build(restore_checkpoint=10000, restore_model_params=True, restore_optimizer_params=True,
              restore_ema_variables=False, verbose=True)

## Evaluation

In [ ]:
runtime.validate(EVAL_BATCH_SIZE)

In [ ]:
runtime.test(EVAL_BATCH_SIZE)

### Random prediction
Either as **binary** (like in training) or **float** (as in raw dataset)...

In [ ]:
AS_BINARY = True
dataset_valid = tt.datasets.moving_mnist.MovingMNISTValidDataset(DATA_DIR,
                                                                 input_shape=[INPUT_SEQ_LENGTH, 64, 64, 1],
                                                                 target_shape=[OUTPUT_SEQ_LENGTH, 64, 64, 1],
                                                                 as_binary=AS_BINARY)
dataset_test = tt.datasets.moving_mnist.MovingMNISTTestDataset(DATA_DIR,
                                                               input_shape=[INPUT_SEQ_LENGTH, 64, 64, 1],
                                                               target_shape=[OUTPUT_SEQ_LENGTH, 64, 64, 1],
                                                               as_binary=AS_BINARY)
runtime.register_datasets(None, dataset_valid, dataset_valid)

In [ ]:
def write_animation(dir_path, inputs, targets, predictions, fps):
    concat_y = np.concatenate((x, y))
    concat_pred = np.concatenate((x, pred))

    tt.utils.video.write_multi_gif(os.path.join(dir_path, "anim-{:02d}.gif".format(i)),
                                   [concat_y, concat_pred],
                                   fps=fps, pad_value=1.0)

    tt.utils.video.write_multi_image_sequence(os.path.join(dir_path, "timeline-{:02d}.png".format(i)),
                                              [concat_y, concat_pred],
                                              pad_value=1.0)

def show(inputs, targets, predictions):
    tt.visualization.display_batch(inputs, ncols=5, nrows=2, title="Inputs")
    tt.visualization.display_batch(targets, ncols=5, nrows=2, title="Targets")
    tt.visualization.display_batch(predictions, ncols=5, nrows=2, title="Predictions")

In [ ]:
dir_path = os.path.join(runtime.train_dir, OUT_DIR_NAME)

inputs, targets = dataset_train.get_batch(NUM_SAMPLES)

predictions = runtime.predict(inputs)

show(inputs[0], targets[0], predictions[0])
for i in range(inputs.shape[0]):
    write_animation(dir_path, inputs[i], targets[i], predictions[i], GIF_FPS)

### Specific Predictions
We are using the inputs used in _Unsupervised Learning with LSTMs_ cropped out of the paper. These consist of two normal sequences, one sequence with only one character and one sequence with three characters...

In [ ]:
SOURCE_PATH = "assets/moving_mnist/"

In [ ]:
runtime.unregister_datasets()
runtime.build(input_shape=[None], target_shape=None)

In [ ]:
def read_sequence(dir_path, seq_id):
    image_list = []
    for i in range(INPUT_SEQ_LENGTH):
        image_path = os.path.join(dir_path, str(seq_id), "{:2d}.png".format(i))
        image = tt.utils.image.read(image_path, color_flags = cv2.IMREAD_GRAYSCALE)
        image_list.append(image)
    seq = np.array(image_list) 
    return seq[:INPUT_SEQ_LENGTH] , seq[INPUT_SEQ_LENGTH:] 

In [ ]:
dir_path = os.path.join(runtime.train_dir, OUT_DIR_NAME)

for i in range(4):
    inputs, targets = read_sequence(SOURCE_PATH, i)
    
    predictions = runtime.predict(inputs)
    
    show(inputs, targets, predictions)
    write_animation(dir_path, inputs, targets, predictions, GIF_FPS)